<a href="https://colab.research.google.com/github/aso1901102/AI_tech2020/blob/master/2020AI0305_fishvideo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 画像を学習して動画解析に利用する

### 画像をDLして解凍

In [1]:
import urllib.request as req

#　魚が映っているファイル
url = "https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.zip"
req.urlretrieve(url, save_file)

# 　魚が映っていないファイル
url = "https://github.com/masatokg/sample_photo/raw/master/nofish.zip"
save_file = "nofish.zip"
req.urlretrieve(url, save_file)

#　解答コマンド
!unzip -o -q "./fish.zip"
!unzip -o -q "./nofish.zip"

### 学習の為の定義

In [2]:
import cv2
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
# 画像のサイズ・パスを指定
image_size = (64,32)
path = "/content"
path_fish = path + "/fish"
path_nofish = path + "/nofish"

#画像データ（説明変数）
x = []
#ラベルデータ（目的変数）
y = []

print()

### 学習に使う自作関数を定義

In [4]:
# 画像データを読み込んんで説明変数、目的変数に追加する処理の関数を定義
def read_dir(path, label, x, y, image_size):
  #　正規表現にマッチするファイルパスリストを取得する
  files = glob.glob(path + "/*.jpg")
  print(files)

  for f in files:
    img = cv2.imread(f)
    img = cv2.resize(img, image_size)
    img_data = img.reshape(-1,)# 行数任意の一次元配列に変換
    x.append(img_data)         #　画像データを説明変数に追加
    y.append(label)           #画像データと同じ順番位置でラベル値を目的半数に追加

## 学習処理

In [5]:
read_dir(path_nofish, 0, x, y, image_size)# ラベルを０：falseで登録
read_dir(path_fish, 1, x, y, image_size)  # ラベルを1 :Tureで登録

# データを分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# ランダムフォレストで学習モデルを生成、学習させる
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
print("学習精度：",accuracy_score(y_test, y_pred))

# 学習モデルのメモリ状態をファイルに保存
with open("fish.pkl", "wb") as fp:
  pickle.dump(clf, fp)

['/content/nofish/124.jpg', '/content/nofish/3648.jpg', '/content/nofish/3656.jpg', '/content/nofish/4.jpg', '/content/nofish/279.jpg', '/content/nofish/30.jpg', '/content/nofish/3451.jpg', '/content/nofish/3466.jpg', '/content/nofish/45.jpg', '/content/nofish/254.jpg', '/content/nofish/5.jpg', '/content/nofish/3467.jpg', '/content/nofish/12.jpg', '/content/nofish/3510.jpg', '/content/nofish/15.jpg', '/content/nofish/245.jpg', '/content/nofish/282.jpg', '/content/nofish/3588.jpg', '/content/nofish/257.jpg', '/content/nofish/25.jpg', '/content/nofish/249.jpg', '/content/nofish/3580.jpg', '/content/nofish/3601.jpg', '/content/nofish/3643.jpg', '/content/nofish/3462.jpg', '/content/nofish/26.jpg', '/content/nofish/32.jpg', '/content/nofish/3596.jpg', '/content/nofish/3597.jpg', '/content/nofish/41.jpg', '/content/nofish/3564.jpg', '/content/nofish/3469.jpg', '/content/nofish/19.jpg', '/content/nofish/3506.jpg', '/content/nofish/242.jpg', '/content/nofish/39.jpg', '/content/nofish/22.jpg',

### 学習モデルを利用して動画から魚の画像を検出

### 解析の準備

In [6]:
import cv2, os, copy, pickle
from google.colab.patches import cv2_imshow

# 学習済みモデルファイルをプログラムに読み込む
with open("fish.pkl","rb") as fp:
  clf = pickle.load(fp)

# 解析画像出力ディレクトリ用文字列
output_dir = "./bestshot"

# 画像出力ディレクトリがなければ作成
if not os.path.isdir(output_dir):
  os.mkdir(output_dir)
  # 以下も同様処理
  # os.makedirs(output_dir, exist_ok=True)
  #

### 動画ファイルの読み込み

In [7]:
url = "https://github.com/masatokg/sample_photo/raw/master/fish.mp4"
save_file = "fish.mp4"
req.urlretrieve(url, save_file)

('fish.mp4', <http.client.HTTPMessage at 0x7f3e21e4e9e8>)

### 動画をプログラムに読み込んで、フレーム差分により変化を検出する

In [8]:
cap = cv2.VideoCapture(save_file)
fish_th = 3
img_last = None
frame_count = 0
count = 0
#読み込むフレームがなくなるまでループ
while True:
  is_ok, frame = cap.read()
  # print(is_ok)
  if not is_ok:
    print("end")
    break
  
  # 差分検出対象フレームのサイズを統一
  frame = cv2.resize(frame,(640,340))

  # 検出エリア枠表示用の画像データを作る
  frame2 = copy.copy(frame)
  frame_count += 1

  # 前のフレームと比較するためにグレースケール、ブラック/ホワイトに変換
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  gray = cv2.GaussianBlur(gray, (15,15), 0)
  img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

  #　前の画像がない（最初の画像）
  if not img_last is None:
    frame_diff = cv2.absdiff(img_last, img_b)
    #　差分のリストを取得
    cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    # 格差分領域に魚が映っているか調べる
    fish_count = 0
    for pt in cnts:
      print(pt)
      x, y, w, h = cv2.boundingRect(pt)

      if w<100 or w > 500: continue

      # 抽出した差分領域に魚が映っているか推論するためにサイズをそろえる
      imgex = frame[y:y+h, x:x+w]
      imgex = cv2.resize(imgex, (64,32))#学習モデルのサイズに合わせる
      image_data = imgex.reshape(-1,) #推論の為の一次元配列に変換

      pred_y = clf.predict([image_data])
      print(pred_y[0])

      #　魚と判定出来たら領域に枠を付けてframe2にする
      if pred_y[0] == 1:
        fish_count += 1
        cv2.rectangle(frame2, (x, y), (x+w, y+h),(0,255,0),2)#緑色の枠を描画

    #　指定サイズ以上の魚が映っていれば画像として保存
    if fish_count > fish_th:
      fname = output_dir + "/fish" + str(count) + ".jpg"
      cv2.imwrite(fname, frame)
      count += 1
      print(fname)

    img_last = img_b
cap.release()
cv2.destroyAllWindows()
print("ok", count, "/", frame_count)

end
ok 0 / 1989


In [9]:
! zip -r /content/bestshot/.zip /content/bestshot/

# 圧縮したzipファイルをDL
from google.colab import files
files.download("/content/bestshot.zip")

updating: content/bestshot/ (stored 0%)


FileNotFoundError: ignored